In [1]:
import pandas as pd
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

In [2]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.impute import SimpleImputer
from sklearn import neighbors, datasets, preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [7]:
data=pd.read_csv(r"/Telco-Customer-Churn.csv")
data
df=pd.DataFrame(data)

df = df.drop('customerID', axis=1)
df=df.dropna()


In [8]:
rows_with_no_phone_service = df[df.apply(lambda row: 'No phone service' in row.values, axis=1)]
df = df.drop(rows_with_no_phone_service.index)

# Identify and remove rows where 'No internet service' is present in any column
rows_with_no_internet_service = df[df.apply(lambda row: 'No internet service' in row.values, axis=1)]
df = df.drop(rows_with_no_internet_service.index)

# Resetting the index after dropping rows
df = df.reset_index(drop=True)

# Display the modified DataFrame
print(df)

      gender  SeniorCitizen Partner Dependents  tenure PhoneService  \
0       Male              0      No         No      34          Yes   
1       Male              0      No         No       2          Yes   
2     Female              0      No         No       2          Yes   
3     Female              0      No         No       8          Yes   
4       Male              0      No        Yes      22          Yes   
...      ...            ...     ...        ...     ...          ...   
4830    Male              0      No         No      19          Yes   
4831    Male              0     Yes        Yes      24          Yes   
4832  Female              0     Yes        Yes      72          Yes   
4833    Male              1     Yes         No       4          Yes   
4834    Male              0      No         No      66          Yes   

     MultipleLines InternetService OnlineSecurity OnlineBackup  \
0               No             DSL            Yes           No   
1              

In [9]:
response_values = ['No', 'Yes', 'no', 'yes']
filtered_columns = df.columns[df.isin(response_values).all()].tolist()
print("Filtered Columns:", filtered_columns)

Filtered Columns: ['Partner', 'Dependents', 'PhoneService', 'MultipleLines', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'PaperlessBilling', 'Churn']


In [10]:
replacement_mapping = {'Yes': 1, 'No': 0}

# Replace values in the specified columns using .loc
df.loc[:, filtered_columns] = df.loc[:, filtered_columns].replace(replacement_mapping)
df[filtered_columns] = df[filtered_columns].apply(pd.to_numeric, errors='coerce')

# Display the resulting DataFrame
print("DataFrame after replacing 'Yes' with 1 and 'No' with 0:")
print(df)

DataFrame after replacing 'Yes' with 1 and 'No' with 0:
      gender  SeniorCitizen  Partner  Dependents  tenure  PhoneService  \
0       Male              0        0           0      34             1   
1       Male              0        0           0       2             1   
2     Female              0        0           0       2             1   
3     Female              0        0           0       8             1   
4       Male              0        0           1      22             1   
...      ...            ...      ...         ...     ...           ...   
4830    Male              0        0           0      19             1   
4831    Male              0        1           1      24             1   
4832  Female              0        1           1      72             1   
4833    Male              1        1           0       4             1   
4834    Male              0        0           0      66             1   

      MultipleLines InternetService  OnlineSecurity  On

<ipython-input-10-ef666e7e63f4>:4: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.loc[:, filtered_columns] = df.loc[:, filtered_columns].replace(replacement_mapping)


In [11]:
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')
# Identify columns with only numerical values
numerical_columns = df.select_dtypes(include='number').columns

# Display the columns with only numerical values
print("Columns with only numerical values:", numerical_columns)

df['TotalCharges']

Columns with only numerical values: Index(['SeniorCitizen', 'Partner', 'Dependents', 'tenure', 'PhoneService',
       'MultipleLines', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
       'TechSupport', 'StreamingTV', 'StreamingMovies', 'PaperlessBilling',
       'MonthlyCharges', 'TotalCharges', 'Churn'],
      dtype='object')


0       1889.50
1        108.15
2        151.65
3        820.50
4       1949.40
         ...   
4830    1495.10
4831    1990.50
4832    7362.90
4833     306.60
4834    6844.50
Name: TotalCharges, Length: 4835, dtype: float64

In [12]:
df['TotalCharges'].isna().sum()

3

In [13]:
df['TotalCharges'].fillna(0,inplace=True)

In [14]:
df['TotalCharges'].isna().sum()

0

In [15]:
#one hot encoding

non_integer_columns = df.select_dtypes(exclude='number').columns.tolist()
non_integer_columns

['gender', 'InternetService', 'Contract', 'PaymentMethod']

In [16]:
data_encoded = pd.get_dummies(df, columns=non_integer_columns)
data_encoded.head()

,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,...,gender_Male,InternetService_DSL,InternetService_Fiber optic,Contract_Month-to-month,Contract_One year,Contract_Two year,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check
0,0,0,0,34,1,0,1,0,1,0,...,1,1,0,0,1,0,0,0,0,1
1,0,0,0,2,1,0,1,1,0,0,...,1,1,0,1,0,0,0,0,0,1
2,0,0,0,2,1,0,0,0,0,0,...,0,0,1,1,0,0,0,0,1,0
3,0,0,0,8,1,1,0,0,1,0,...,0,0,1,1,0,0,0,0,1,0
4,0,0,1,22,1,1,0,1,0,0,...,1,0,1,1,0,0,0,1,0,0


In [17]:
df_boolean = data_encoded.select_dtypes(include='bool')
data_encoded[df_boolean.columns] = df_boolean.applymap(lambda x: int(x))
column_names = data_encoded.columns.tolist()

# Print the list of column names
print("Column Names:", column_names)

Column Names: ['SeniorCitizen', 'Partner', 'Dependents', 'tenure', 'PhoneService', 'MultipleLines', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'PaperlessBilling', 'MonthlyCharges', 'TotalCharges', 'Churn', 'gender_Female', 'gender_Male', 'InternetService_DSL', 'InternetService_Fiber optic', 'Contract_Month-to-month', 'Contract_One year', 'Contract_Two year', 'PaymentMethod_Bank transfer (automatic)', 'PaymentMethod_Credit card (automatic)', 'PaymentMethod_Electronic check', 'PaymentMethod_Mailed check']


In [18]:
target_column = 'Churn'
columns_order = [col for col in data_encoded.columns if col != target_column]

# Append the target column to the end of the list
columns_order.append(target_column)

# Reorder the DataFrame columns based on the new order
data_encoded= data_encoded[columns_order]

# Display the updated DataFrame
print(data_encoded)

      SeniorCitizen  Partner  Dependents  tenure  PhoneService  MultipleLines  \
0                 0        0           0      34             1              0   
1                 0        0           0       2             1              0   
2                 0        0           0       2             1              0   
3                 0        0           0       8             1              1   
4                 0        0           1      22             1              1   
...             ...      ...         ...     ...           ...            ...   
4830              0        0           0      19             1              0   
4831              0        1           1      24             1              1   
4832              0        1           1      72             1              1   
4833              1        1           0       4             1              1   
4834              0        0           0      66             1              0   

      OnlineSecurity  Onlin

In [19]:
features = data_encoded.iloc[:, :-1]
target = data_encoded.iloc[:, -1]

In [20]:
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.3, random_state=42)

In [21]:
nan_values = X_train.isna().sum()
print(nan_values)

SeniorCitizen                              0
Partner                                    0
Dependents                                 0
tenure                                     0
PhoneService                               0
MultipleLines                              0
OnlineSecurity                             0
OnlineBackup                               0
DeviceProtection                           0
TechSupport                                0
StreamingTV                                0
StreamingMovies                            0
PaperlessBilling                           0
MonthlyCharges                             0
TotalCharges                               0
gender_Female                              0
gender_Male                                0
InternetService_DSL                        0
InternetService_Fiber optic                0
Contract_Month-to-month                    0
Contract_One year                          0
Contract_Two year                          0
PaymentMet

In [22]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled= scaler.transform(X_test)

In [24]:
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.metrics import accuracy_score
# Support Vector Machine (SVM)
svm_model = SVC()
svm_model.fit(X_train_scaled, y_train)
svm_predictions = svm_model.predict(X_test_scaled)
svm_accuracy = accuracy_score(y_test, svm_predictions)
print(f"SVM Accuracy: {svm_accuracy}")

# Decision Tree
dt_model = DecisionTreeClassifier()
dt_model.fit(X_train_scaled, y_train)
dt_predictions = dt_model.predict(X_test_scaled)
dt_accuracy = accuracy_score(y_test, dt_predictions)
print(f"Decision Tree Accuracy: {dt_accuracy}")

# Random Forest
rf_model = RandomForestClassifier()
rf_model.fit(X_train_scaled, y_train)
rf_predictions = rf_model.predict(X_test_scaled)
rf_accuracy = accuracy_score(y_test, rf_predictions)
print(f"Random Forest Accuracy: {rf_accuracy}")

# Boosting Algorithm (AdaBoost)
boosting_model = AdaBoostClassifier()
boosting_model.fit(X_train_scaled, y_train)
boosting_predictions = boosting_model.predict(X_test_scaled)
boosting_accuracy = accuracy_score(y_test, boosting_predictions)
print(f"Boosting Algorithm (AdaBoost) Accuracy: {boosting_accuracy}")

SVM Accuracy: 0.7518952446588559
Decision Tree Accuracy: 0.6574776016540317
Random Forest Accuracy: 0.7388008270158511
Boosting Algorithm (AdaBoost) Accuracy: 0.7518952446588559


In [25]:
svm_model = SVC(C=1.0, kernel='rbf')
dt_model = DecisionTreeClassifier(max_depth=5, min_samples_split=2, min_samples_leaf=1)
rf_model = RandomForestClassifier(n_estimators=100, max_depth=10, max_features='sqrt')
boosting_model = AdaBoostClassifier(n_estimators=50, learning_rate=1.0, base_estimator=DecisionTreeClassifier(max_depth=1))


In [27]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.svm import SVC
from scipy.stats import uniform

# Tunable Parameters
svm_param_grid = {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf'], 'gamma': ['scale', 'auto']}

# GridSearchCV
svm_grid_search = GridSearchCV(SVC(), svm_param_grid, cv=3)
svm_grid_search.fit(X_train_scaled, y_train)
svm_best_params = svm_grid_search.best_params_
print("SVM Best Parameters:", svm_best_params)

# RandomizedSearchCV
svm_param_dist = {'C': uniform(0.1, 10), 'kernel': ['linear', 'rbf'], 'gamma': ['scale', 'auto']}
svm_random_search = RandomizedSearchCV(SVC(), svm_param_dist, n_iter=10, cv=3)
svm_random_search.fit(X_train_scaled, y_train)
svm_best_params_random = svm_random_search.best_params_
print("SVM Best Parameters (Randomized):", svm_best_params_random)


SVM Best Parameters: {'C': 10, 'gamma': 'scale', 'kernel': 'linear'}
SVM Best Parameters (Randomized): {'C': 3.5092212015339475, 'gamma': 'auto', 'kernel': 'linear'}


In [28]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier
from scipy.stats import randint

# Tunable Parameters
dt_param_grid = {'max_depth': [None, 5, 10], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4]}

# GridSearchCV
dt_grid_search = GridSearchCV(DecisionTreeClassifier(), dt_param_grid, cv=3)
dt_grid_search.fit(X_train_scaled, y_train)
dt_best_params = dt_grid_search.best_params_
print("Decision Tree Best Parameters:", dt_best_params)

# RandomizedSearchCV
dt_param_dist = {'max_depth': [None, 5, 10, 15], 'min_samples_split': randint(2, 10), 'min_samples_leaf': randint(1, 5)}
dt_random_search = RandomizedSearchCV(DecisionTreeClassifier(), dt_param_dist, n_iter=10, cv=3)
dt_random_search.fit(X_train_scaled, y_train)
dt_best_params_random = dt_random_search.best_params_
print("Decision Tree Best Parameters (Randomized):", dt_best_params_random)


Decision Tree Best Parameters: {'max_depth': 5, 'min_samples_leaf': 1, 'min_samples_split': 5}
Decision Tree Best Parameters (Randomized): {'max_depth': 5, 'min_samples_leaf': 2, 'min_samples_split': 8}


In [29]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from scipy.stats import randint

# Tunable Parameters
rf_param_grid = {'n_estimators': [50, 100, 200], 'max_depth': [None, 10, 20], 'max_features': ['sqrt', 'log2']}

# GridSearchCV
rf_grid_search = GridSearchCV(RandomForestClassifier(), rf_param_grid, cv=3)
rf_grid_search.fit(X_train_scaled, y_train)
rf_best_params = rf_grid_search.best_params_
print("Random Forest Best Parameters:", rf_best_params)

# RandomizedSearchCV
rf_param_dist = {'n_estimators': [50, 100, 200], 'max_depth': [None, 10, 20], 'max_features': ['sqrt', 'log2']}
rf_random_search = RandomizedSearchCV(RandomForestClassifier(), rf_param_dist, n_iter=10, cv=3)
rf_random_search.fit(X_train_scaled, y_train)
rf_best_params_random = rf_random_search.best_params_
print("Random Forest Best Parameters (Randomized):", rf_best_params_random)


Random Forest Best Parameters: {'max_depth': 10, 'max_features': 'log2', 'n_estimators': 100}
Random Forest Best Parameters (Randomized): {'n_estimators': 100, 'max_features': 'log2', 'max_depth': 10}


In [45]:
# Import necessary libraries
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

# Define the parameter grids
param_grid = {
    'n_estimators': [50, 100, 150, 200],
    'learning_rate': [0.1, 0.5, 1.0],
    'algorithm': ['SAMME', 'SAMME.R']
}

# Initialize the model
clf = AdaBoostClassifier()

# Perform GridSearchCV
grid_search = GridSearchCV(clf, param_grid, cv=5)
grid_search.fit(X_train_scaled, y_train)

# Get the best parameters
best_params_grid = grid_search.best_params_

# Perform RandomizedSearchCV
random_search = RandomizedSearchCV(clf, param_grid, cv=5, n_iter=10)
random_search.fit(X_train_scaled, y_train)

# Get the best parameters
best_params_random = random_search.best_params_

# Print the best parameters for both searches
print("Best parameters for GridSearchCV:", best_params_grid)
print("Best parameters for RandomizedSearchCV:", best_params_random)

Best parameters for GridSearchCV: {'algorithm': 'SAMME.R', 'learning_rate': 0.5, 'n_estimators': 100}
Best parameters for RandomizedSearchCV: {'n_estimators': 100, 'learning_rate': 0.5, 'algorithm': 'SAMME.R'}
